In [3]:
%pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 10.5 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 6.0 MB/s eta 0:00:0012.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 9.6 MB/s eta 0:00:000m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 9.6 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 7.4 MB/s eta 0:00:008 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import mlflow


tracking_uri = "./mlruns"

mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("hulo")

for i in range(32):
    with mlflow.start_run(run_name= f"run_{i}"):
        mlflow.log_param("model_name", "my_model")
        mlflow.log_param("layer_count", 3)
        mlflow.log_param("loss_function", "cross_entropy")
        mlflow.log_metric("training_time", 121.0)
        mlflow.log_metric("accuracy", 120.0)

In [7]:
import pandas as pd

def mlflow_all_experiments(tracking_uri="./mlruns"):
    client = mlflow.tracking.MlflowClient(tracking_uri=tracking_uri)
    experiment_ids = [e.experiment_id for e in client.search_experiments()]
    records = []
    for id in experiment_ids:
        records.extend(mlflow_experiment_records(id, client))
    df = pd.DataFrame.from_records(records)
    return df

def mlflow_experiment(name: str, tracking_uri="./mlruns") -> pd.DataFrame | None:
    client = mlflow.tracking.MlflowClient(tracking_uri=tracking_uri)
    experiment_ids = [e.experiment_id for e in client.search_experiments(filter_string=f"name = '${name}'")]
    if len(experiment_ids) == 0:
        return None
    records = mlflow_experiment_records(experiment_ids[0], client)
    df = pd.DataFrame.from_records(records)
    return df

def mlflow_experiment_records(id: str, client: mlflow.client.MlflowClient) -> dict:
    records = []
    exp = client.get_experiment(id)
    runs = client.search_runs(id)
    ex_record = dict()
    for (k,v) in exp:
        ex_record[f"ex_{k}"] = v

    for r in runs:
        record = {**ex_record}
        for k in r.data.params:
            record[f"p_{k}"] = r.data.params[k]
        for k in r.data.metrics:
            record[f"m_{k}"] = r.data.metrics[k]
        for k in r.data.tags:
            record[f"tag_{k}"] = r.data.tags[k]
        records.append(record)
    return records
df = mlflow_all_experiments()
df[["ex_name", "p_layer_count", "p_model_name", "m_training_time", "m_accuracy"]]


,ex_name,p_layer_count,p_model_name,m_training_time,m_accuracy
0,hulo,3,my_model,121.0,120.0
1,hulo,3,my_model,121.0,120.0
2,hulo,3,my_model,121.0,120.0
3,hulo,3,my_model,121.0,120.0
4,hulo,3,my_model,121.0,120.0
...,...,...,...,...,...
62,hulo,3,my_model,121.0,120.0
63,hulo,3,my_model,121.0,120.0
64,exp777,3,my_model,121.0,120.0
65,exp777,3,my_model,121.0,120.0
